# Track B Pipeline Orchestration
This notebook coordinates the audit, feature integration, and modeling scripts. Use the dry-run workflow first to validate configuration before triggering the full pipeline.

In [ ]:
from pathlib import Path
import json
import os
import subprocess
from typing import Sequence

REPO_ROOT = Path("/workspaces/bitcore").resolve()
PIPELINE_SCRIPT = REPO_ROOT / "workspace" / "scripts" / "run_pipeline.py"
DRY_RUN_DEFAULT = True

if DRY_RUN_DEFAULT:
    os.environ["BITCORE_DRY_RUN"] = "1"
else:
    os.environ.pop("BITCORE_DRY_RUN", None)

def run_command(command: Sequence[str], cwd: Path | None = None) -> subprocess.CompletedProcess:
    resolved_cwd = cwd or REPO_ROOT
    result = subprocess.run(
        list(command),
        cwd=resolved_cwd,
        text=True,
        capture_output=True,
        check=False,
    )
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print(result.stderr)
    if result.returncode != 0:
        raise RuntimeError(f"Command {command} failed with exit code {result.returncode}")
    return result

In [ ]:
plan_process = run_command(["python", str(PIPELINE_SCRIPT), "--plan"])
plan = json.loads(plan_process.stdout)
plan

In [ ]:
if DRY_RUN_DEFAULT:
    run_command(["python", str(PIPELINE_SCRIPT), "--dry-run"])
else:
    print("Dry-run disabled; adjust DRY_RUN_DEFAULT in the configuration cell if required.")

## Execute Full Pipeline
Switch `DRY_RUN_DEFAULT` to `False` and re-run the configuration cell before executing this block.

In [ ]:
if not DRY_RUN_DEFAULT:
    run_command(["python", str(PIPELINE_SCRIPT)])
else:
    print("Full pipeline execution is gated behind DRY_RUN_DEFAULT.")